In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
import yasa
from preprocessing import crop_hypno, extract_features

In [19]:
# Define paths (can be defined in config files)
eeg_dir = 'data/edfs/shhs2/'
hypno_dir = 'data/annotations-events-profusion/shhs2/'
parent_dir = os.path.dirname(os.getcwd())
out_dir = './output/features/'

In [3]:
df_subj = pd.read_csv("./output/split/shhs_split.csv")
df_subj = df_subj.query("set == 'training'").set_index("subj")

In [4]:
print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

2 subjects remaining


,age,male,bmi,ahi,ethnicity,set,hypertension
subj,,,,,,,
200077,41.0,1,23.388687,9.738220,caucasian,training,0.0
200078,54.0,1,30.211833,19.685039,caucasian,training,1.0


In [5]:
df = []
include = ['EEG', 'EOG(L)', 'EMG']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'shhs2-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'shhs2-' + str(sub) + '-profusion.xml'
    
    # Check that file exists
    if not os.path.isfile(eeg_file):
        warnings.warn("File not found %s" % eeg_file)
        continue
    if not os.path.isfile(hypno_file):
        warnings.warn("File not found %s" % hypno_file)
        continue

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
        # Skip subjects if channel were not found
        assert len(raw.ch_names) == len(include)
        raw.load_data()
    except:
        continue
        
    # Resample and high-pass filter 
    raw.resample(sf, npad="auto")
    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # (Optional) We keep up to 15 minutes before / after sleep
    # hypno, tmin, tmax = crop_hypno(hypno)
    # raw.crop(tmin, tmax)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA (DISABLED)
    # Hypnogram must include all stages
#     if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
#         print("- Not all stages are present.")
#         continue
#     # If the duration is not between 4 to 12 hours, skip subject
#     if not(4 < n_epochs / 120 < 12):
#         print("- Recording too short/long.")
#         continue

    # EXTRACT FEATURES
    features = extract_features(df_subj, sub, raw, include)
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [6]:
# Convert to category
df['stage'] = df['stage'].astype('category')

In [24]:
# Export
df.to_parquet(out_dir + "features_nsrr_shhs2.parquet")

## Stats

In [8]:
df['stage'].value_counts(normalize=True, sort=True)

W     0.424048
N2    0.384093
R     0.147003
N1    0.028270
N3    0.016585
Name: stage, dtype: float64

In [9]:
df.groupby('stage')['eeg_iqr'].median()

stage
N1     11.011239
N2     14.550829
N3     20.749241
R       9.838191
W     231.408218
Name: eeg_iqr, dtype: float32